This will handle the first step of the process - creating the code from the images.

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
from glob import glob
import private_secrets as secrets

In [10]:
# basic setup and auth
subscription_key = secrets.SUB_KEY
endpoint = "https://test-image-rec.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
GH_BASE = 'https://raw.githubusercontent.com/GitToby/college-code-reincarnation/master/meta_work/img/'

In [12]:
# fetch all images
images = glob("img/*")

# replace the folder to get just the image filenames
for i in range(len(images)):
    images[i] = images[i].replace("img/","")

image_urls = [GH_BASE + im for im in images]

print(len(images), "Images")

28 Images


In [15]:
def make_request(img_url: str, print_res: bool = False) -> str:
    # Make initial request
    recognize_handw_results = computervision_client.read(img_url, raw=True)

    # Parse response to get op ID 
    operation_location_remote = recognize_handw_results.headers["Operation-Location"]
    operation_id = operation_location_remote.split("/")[-1]
    
    # Await the response while remote does processing (nast impl but I dont care) 
    while True:
        get_handw_text_results = computervision_client.get_read_result(operation_id)
        if get_handw_text_results.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    # Print the detected text, line by line
    if get_handw_text_results.status == OperationStatusCodes.succeeded:
        for text_result in get_handw_text_results.analyze_result.read_results:
            if print_res:
                for line in text_result.lines:
                    print(line.text)
            return text_result.lines

In [33]:
def pipe_image_to_txt(image_file_path: str) -> (str,str):
    # form known url
    image_url = GH_BASE + image_file_path

    # Make request and get data
    res = make_request(image_url)

    # Join resulting data to a string
    out_text = "\n".join(t.text for t in res)
    
    # dump into an out file
    out_file_name = images[0].replace(".jpg",".txt")
    with open(f"out/{out_file_name}", "w") as f:
        f.write(out_text)

    # util result
    return image_file_path, out_file_name


In [34]:
for i, image_file_path in enumerate(images[:1]):
    original, test_file = pipe_image_to_txt(image_file_path)
    print(i, ,"|", original, "->", test_file)

0 20201114_120404.jpg 20201114_120404.txt
